### Imports

In [27]:
spark.version

u'2.2.0'

In [2]:
import re
import datetime
from pyspark.sql.functions import *
import pandas as pd
pd.set_option('display.max_colwidth', 80)

### Add Lookup Tables

In [3]:
monthMap = {
    "Jan": "01",
    "Feb": "02",
    "Mar": "03",
    "Apr": "04",
    "May": "05",
    "Jun": "06",
    "Jul": "07",
    "Aug": "08",
    "Sep": "09",
    "Oct": "10",
    "Nov": "11",
    "Dec": "12"
}

teamMap = {
  "Atlanta" : "atl",
  "Boston"  : "bos",
  "Brooklyn"  : "bkn",
  "Charlotte"  : "cha",
  "Chicago"  : "chi",
  "Cleveland"  : "cle",
  "Dallas"  : "dal",
  "Denver"  : "den",
  "Detroit"  : "det",
  "Golden State"  : "gst",
  "Houston"  : "hou",
  "Indiana"  : "ind",
  "LA Clippers"  : "lac",
  "LA Lakers"  : "lal",
  "Memphis"  : "mem",
  "Miami"  : "mia",
  "Milwaukee"  : "mil",
  "Minnesota"  : "min",
  "New Orleans"  : "nor",
  "New York"  : "nyk",
  "Oklahoma City"  : "okc",
  "Orlando"  : "orl",
  "Philadelphia"  : "phi",
  "Phila."  : "phi",
  "Phoenix"  : "pho",
  "Portland"  : "por",
  "Sacramento" : "sac",
  "San Antonio"  : "san",
  "Toronto"  : "tor",
  "Utah"  : "uta",
  "Washington"  : "wsh",
   None : "none"}

### Load In NBA Score Data Set

In [4]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType,DateType

customSchema = StructType([
    StructField("dateOrig", DateType()),
    StructField("ts", StringType()),
    StructField("teamlonga", StringType()),
    StructField("scorea", IntegerType()),
    StructField("teamlongb", StringType()),
    StructField("scoreb", IntegerType()),
    StructField("timestring", StringType()),
    StructField("timeleft", DoubleType()),
    StructField("gameid", IntegerType())
])

nbafile = '/data2/nba-rt-prediction/scoredata/scores_nba.2015.test.dat'
rtscoresAndFinalDF = spark.read.format('csv')\
                    .option("header", "false")\
                    .option("inferSchema", "false")\
                    .option("nullValue", "empty")\
                    .option("dateFormat", "yyyy-MM-dd")\
                    .option("mode","DROPMALFORMED")\
                    .schema(customSchema)\
                    .load(nbafile).coalesce(2)
    
rtscoresAndFinalDF.show(5)


+----------+--------+-------------+------+------------+------+------------+--------+---------+
|  dateOrig|      ts|    teamlonga|scorea|   teamlongb|scoreb|  timestring|timeleft|   gameid|
+----------+--------+-------------+------+------------+------+------------+--------+---------+
|2016-04-05|15:06:16|      Phoenix|     0|     Atlanta|     0|(8:00 PM ET)|    48.0|400829044|
|2016-04-05|15:06:16|      Chicago|     0|     Memphis|     0|(8:00 PM ET)|    48.0|400829045|
|2016-04-05|15:06:16|    Cleveland|     0|   Milwaukee|     0|(8:00 PM ET)|    48.0|400829046|
|2016-04-05|15:06:16|Oklahoma City|     0|      Denver|     0|(9:00 PM ET)|    48.0|400829047|
|2016-04-05|15:06:16|  New Orleans|     0|Philadelphia|     0|(7:00 PM ET)|    48.0|400829041|
+----------+--------+-------------+------+------------+------+------------+--------+---------+
only showing top 5 rows



### UDFs For Creating Extra Columns In Real Time Data Frame

In [5]:
# Create new team name column.. do simple lookup conversion with a UDF
def mapper(teamin) :
    return teamMap[teamin]

mapperudf = udf(mapper)


In [6]:
# Date Logic to adjust for games that finish on the day after .... 
# This is so that I can join them against the spread which was dated the day prior...
# This is due to not having a great key to join my tables ...

datecrossregex = re.compile("^0[0-3]") # midnight to 3am
def dateadjust(datein, tsin ) : 
    #dateary = datein.split("-")
    tsary   = tsin.split(":")
    sub_one_day = datetime.timedelta(days=1)
    newdate = datein
    if datecrossregex.match(tsary[0]) :
        #day = "%02d".format(int(dateary[2]) -1)
        #newdate = dateary(0) + "-" + dateary(1) + "-" + day   
        newdate = datein - sub_one_day
    return str(newdate)

dateadjustudf = udf(dateadjust)


# UDFs to create some extra features ... this one is for an experiemental combination of Time left and Score difference.  
# Made this via intuition.  This can be extended to add other custom features
import math
def scoredivtimeXform(numerator, denominator):
    rv = numerator/(math.pow(denominator+1,0.5))
    return rv
scoredivtimeUdf = udf(scoredivtimeXform)


### Wrangle The Real Time And Final Score Data.  Add Columns To The Data Set

In [7]:
# Remove Overtime games from this analysis
rtscoresAndFinalDF = rtscoresAndFinalDF.filter(~col("timestring").like("%OT%"))

# Create short 3 character team names 
rtscoresAndFinalDF = rtscoresAndFinalDF.withColumn("teama", mapperudf(col("teamlonga")))
rtscoresAndFinalDF = rtscoresAndFinalDF.withColumn("teamb", mapperudf(col("teamlongb")))

# Add a score differential Column 
rtscoresAndFinalDF = rtscoresAndFinalDF.withColumn("scorea-scoreb", col("scorea") - col("scoreb"))

# Transform the Date.  This is for games that spanned multiple days and gave me a headache.  
# Games adjusted to the day they started on.
rtscoresAndFinalDF = rtscoresAndFinalDF.withColumn("date",  dateadjustudf(col("dateOrig"),col("ts")))

# Create a Key for me to use to join with my odds data later.  Key = date.teama.teamb
rtscoresAndFinalDF = rtscoresAndFinalDF.withColumn("key", concat(col("date"),lit("."),col("teama"),lit("."),col("teamb")))



In [9]:
rtscoresAndFinalDF.show(5)


+----------+--------+-------------+------+------------+------+-------------+--------+---------+-----+-----+-------------+----------+------------------+
|  dateOrig|      ts|    teamlonga|scorea|   teamlongb|scoreb|   timestring|timeleft|   gameid|teama|teamb|scorea-scoreb|      date|               key|
+----------+--------+-------------+------+------------+------+-------------+--------+---------+-----+-----+-------------+----------+------------------+
|2016-04-05|15:06:16|      Phoenix|     0|     Atlanta|     0| (8:00 PM ET)|    48.0|400829044|  pho|  atl|            0|2016-04-05|2016-04-05.pho.atl|
|2016-04-05|15:06:16|      Chicago|     0|     Memphis|     0| (8:00 PM ET)|    48.0|400829045|  chi|  mem|            0|2016-04-05|2016-04-05.chi.mem|
|2016-04-05|15:06:16|    Cleveland|     0|   Milwaukee|     0| (8:00 PM ET)|    48.0|400829046|  cle|  mil|            0|2016-04-05|2016-04-05.cle.mil|
|2016-04-05|15:06:16|Oklahoma City|     0|      Denver|     0| (9:00 PM ET)|    48.0|400

### Separate The Real Time And Final Data From One Common Dataframe To Two Dataframes

In [10]:
# Currently based on the way the data was sampled, both real time scores and final scores are written as seperate records to the same file.  I need to pull these apart, and then join the dataframes so that I have a real time score and features, and know if the game was won or lost ....

# Create Final Score DF
# Note a shortcut for repeating the dataframe within the filter is to use a $   df.filter(df("foo").contains ... is equiv to df.filter($"foo".contains)

finalscoresDF = rtscoresAndFinalDF.filter(col("timestring").like("%FINAL%"))

# Rename some columns so that join later doesnt have name overlaps
finalscoresDF = finalscoresDF.withColumnRenamed("scorea", "fscorea")
finalscoresDF = finalscoresDF.withColumnRenamed("scoreb", "fscoreb")

# Create final score difference
finalscoresDF = finalscoresDF.withColumn("fscorea-fscoreb", col("fscorea") - col("fscoreb"))

# Add a Win/loss column Win = 1, Loss = 0
finalscoresDF = finalscoresDF.withColumn("win-loss-enc", (when(col("fscorea-fscoreb") > 0.0, 1.0).otherwise(0)))

#(when(df['age'] == 2, 3).otherwise(4)

# Remove Halftime records as this particular case isn't handled well... (for now)
rtscoresDF = rtscoresAndFinalDF.filter(~col("timestring").like("%FINAL%")).filter(~col("timestring").like("HALFTIME"))

# Create final score difference
rtscoresDF = rtscoresDF.withColumn("scorea-scoreb", col("scorea") - col("scoreb"))

# Create a unique feature based on my custom UDF.  Idea here is that I have intuition that timeleft and score difference are a strong predictor when combined
rtscoresDF = rtscoresDF.withColumn("score-div-time", scoredivtimeUdf(col("scorea") - col("scoreb"), col("timeleft")*2))


### Lets Take A Look Of What We Have For The Two Dataframes We Just Wrangled

In [11]:
# Some Printouts .....
print("final scores data frame")
finalscoresDF.show(5)
print("real time scores data frame")
rtscoresDF.show(5)
finalscoresDF.printSchema


print "##########################################"
print "Total Data Points in rtscoresDF = {0}".format(rtscoresDF.count())
print "Total Data Points in rtscoresDF uniq = {0}".format(rtscoresDF.sort("key").distinct().count())
print "Total Data Points in finalscoresDF = {0}".format(finalscoresDF.count())
print "##########################################"




final scores data frame
+----------+--------+-----------+-------+------------+-------+----------+--------+---------+-----+-----+-------------+----------+------------------+---------------+------------+
|  dateOrig|      ts|  teamlonga|fscorea|   teamlongb|fscoreb|timestring|timeleft|   gameid|teama|teamb|scorea-scoreb|      date|               key|fscorea-fscoreb|win-loss-enc|
+----------+--------+-----------+-------+------------+-------+----------+--------+---------+-----+-----+-------------+----------+------------------+---------------+------------+
|2016-04-05|21:22:09|New Orleans|     93|Philadelphia|    107|   (FINAL)|     0.0|400829041|  nor|  phi|          -14|2016-04-05|2016-04-05.nor.phi|            -14|         0.0|
|2016-04-05|22:08:42|  Charlotte|     90|     Toronto|     96|   (FINAL)|     0.0|400829043|  cha|  tor|           -6|2016-04-05|2016-04-05.cha.tor|             -6|         0.0|
|2016-04-05|22:25:25|    Chicago|     92|     Memphis|    108|   (FINAL)|     0.0|4008

### Interpret the Odds data
````How to interpret the odds data ...
Example Golden State -12.5 O (207.0) -125.0 | Detroit 12.5 U (207.0) 145.0
Here Golden State the away team is a 12.5 pt favorite to win.  The over under is in parentheses (207) and is the 50/50 line between teams sum of scores
being above/below that line.  
Finally the -125 / +145 numbers are whats known at the moneyline odds. 
    A negative number means you need to bet 125$ to get a 100$ payout
    A positive number means you need to bet 100$ to get a 145$ payout
```

### Load In Odds Data

In [12]:
# Here, the data is very raw, and needs to be pre-processed .  I will start by loading it as an RDD and perform a lot of transformations.  Once I have it properly parsed, I will convert to a dataframe.
# This is not beautiful, but gets the job done
# Data format .....
#       <title>New Orleans 2.5 O (207.0) 125.0 | Phila. -2.5 U (207.0) -145.0 (Apr 05, 2016 07:10 PM)</title>
#       <title>Detroit 4.0 O (202.0) 160.0 | Miami -4.0 U (202.0) -190.0 (Apr 05, 2016 08:05 PM)</title>
oddsfile = "/data2/nba-rt-prediction/nbaodds_042516.xml"

# Reading the data in as an RDD first.  There isn't a dataframe parser for this XML I have, so I will write a custom parser ....
oddsrdd = spark.read.text(oddsfile).rdd
# just grabbing the text within the < ... > tags.  I can do this, because the format is super simple and not nested
# the subscript [0] is due to the fact that spark.read.text read in the oddsfile of Type Row.  Need to index
# into it to get the string
gameStringRdd = oddsrdd.map(lambda x : x[0][x[0].find('>')+1:x[0].rfind('<')])


In [14]:
# This is where I do the heavy lifting of parsing my XML .. and then finally convert my RDD to a dataframe .....
# just lots of string parsing and data type conversions
def parseOdds(line_in) : 
    away_str = line_in[0:line_in.find('|')]  
    home_and_date_str = line_in[line_in.find('|')+2:-1]
    home_str = home_and_date_str[0:home_and_date_str.rfind('(')]
    date_str = home_and_date_str[home_and_date_str.rfind('(')+1:len(home_and_date_str)]
    date_str = date_str.replace(',', '')
    
    # parse away string
    overunder = away_str[away_str.find('(')+1:away_str.find(')')]
    teamaml = away_str[away_str.find(')')+2:len(away_str)-1]
    away_str_2 = away_str[0:away_str.find('(')-3]
    teamaspread = away_str_2[away_str_2.rfind(' ')+1:len(away_str_2)]
    teamlonga = away_str_2[0:away_str_2.rfind(' ')]
    teama = teamMap[teamlonga]
    
     # parse home string
    overunder = home_str[home_str.find('(')+1:home_str.find(')')]
    teamhml = home_str[home_str.find(')')+2:len(home_str)-1]
    home_str_2 = home_str[0:home_str.find('(')-3]
    teamhspread = home_str_2[home_str_2.rfind(' ')+1:len(home_str_2)]
    teamlongh = home_str_2[0:home_str_2.rfind(' ')]
    teamh = teamMap[teamlongh]
   
    # parse date string
    dateInfo = date_str.split(' ')
    dateStr = dateInfo[2] + "-" + monthMap[dateInfo[0]] + "-" + dateInfo[1]
    # This will become my join key for the other data sets
    key = dateStr +"." + teama + "." + teamh
    return (key,teamlonga,teama,teamaspread,overunder,teamaml,teamlongh,teamh,teamhml,dateStr)

#def parseOdds(line_in) : 
#    away_str = line_in[0:line_in.find('|')]  
#    return (away_str,away_str)


oddsDF = gameStringRdd.map(lambda x : parseOdds(x))\
        .toDF(["key","teamlonga","teama","teamaspread","overunder","teamaml","teamlongh","teamh","teamhml","dateStr"]).distinct()

# OddsDF has some dups due to the fact that I have multiple readings ...
oddsDF.registerTempTable("odds_table")

oddsDF = spark.sql("SELECT key, FIRST(teamlonga) as teamlonga, FIRST(teama) as teama,\
  AVG(teamaspread) as teamaspread, AVG(overunder) as overunder, AVG(teamaml) as teamaml,\
  FIRST(teamlongh) as teamlongh,FIRST(teamh) as teamh,AVG(teamhml) as teamhml, FIRST(dateStr) as dateStr FROM odds_table GROUP BY key")


### Inspect Some Of The Odds Data

In [15]:
oddsDF.show(5)
print "Total Home Teams      = {0}".format(oddsDF.select("teamh").distinct().count())
print "Total Away Teams      = {0}".format(oddsDF.select("teama").distinct().count())
print "Total Games Collected = {0} ".format(oddsDF.count())


+------------------+----------+-----+-----------+---------+-------+-----------+-----+-------+----------+
|               key| teamlonga|teama|teamaspread|overunder|teamaml|  teamlongh|teamh|teamhml|   dateStr|
+------------------+----------+-----+-----------+---------+-------+-----------+-----+-------+----------+
|2016-04-24.atl.bos|   Atlanta|  atl|        1.5|    203.5|    0.0|     Boston|  bos| -120.0|2016-04-24|
|2016-04-19.mem.san|   Memphis|  mem|      18.25|    187.0| -110.0|San Antonio|  san| -110.0|2016-04-19|
|2016-04-26.ind.tor|   Indiana|  ind|        7.0|    192.0|  260.0|    Toronto|  tor| -320.0|2016-04-26|
|2016-04-13.mia.bos|     Miami|  mia|        5.0|    206.5|  170.0|     Boston|  bos| -200.0|2016-04-13|
|2016-04-13.sac.hou|Sacramento|  sac|       15.0|    222.0| -110.0|    Houston|  hou| -110.0|2016-04-13|
+------------------+----------+-----+-----------+---------+-------+-----------+-----+-------+----------+
only showing top 5 rows

Total Home Teams      = 30
Tot

### Join The Odds And Final Score Data Sets

In [16]:
# Here is where we join the Odds/Realtime scores/ Final Scores into one wholistic data set as input for Logistic Machine Learning

# Create a smaller Final Score Dataframe.  Just keep the key, final score a and b, the win/loss indicator
finalslicedscoresDF1 = finalscoresDF.select(["key","fscorea","fscoreb","win-loss-enc"]).distinct()
# First Join the 2 smallest data frames ... odd and final.
gameDF = oddsDF.join(finalslicedscoresDF1, oddsDF["key"] == finalslicedscoresDF1["key"], "inner").drop(oddsDF["key"])
# Drop these redundant columns prior to joining with Realtime score dataframe
gameDF = gameDF.drop("teamlonga")
gameDF = gameDF.drop("teamlongb")
gameDF = gameDF.drop("teama")
gameDF = gameDF.drop("teamb")

In [17]:
# Print Out the Game Dataframe ... notice we have the odds data merged with the win loss data ....
#print("gameDF")
#gameDF.sort(["key"]).show(152)
print "Total finalscoresDF = {0}".format(finalscoresDF.count())
print "Total oddsDF = {0}".format(oddsDF.count())
print "Total Games after joining odds and score data = {0}".format(gameDF.count())

#gameDF.select("key").sort(["key"]).show(144)

finalscoresDF.filter(col("key") == "2016-04-24.gst.hou").show()
oddsDF.filter(col("key") == "2016-04-24.gst.hou").show()


Total finalscoresDF = 116
Total oddsDF = 111
Total Games after joining odds and score data = 101
+----------+--------+------------+-------+---------+-------+----------+--------+---------+-----+-----+-------------+----------+------------------+---------------+------------+
|  dateOrig|      ts|   teamlonga|fscorea|teamlongb|fscoreb|timestring|timeleft|   gameid|teama|teamb|scorea-scoreb|      date|               key|fscorea-fscoreb|win-loss-enc|
+----------+--------+------------+-------+---------+-------+----------+--------+---------+-----+-----+-------------+----------+------------------+---------------+------------+
|2016-04-24|18:29:52|Golden State|    121|  Houston|     94|   (FINAL)|     0.0|400874348|  gst|  hou|           27|2016-04-24|2016-04-24.gst.hou|             27|         1.0|
+----------+--------+------------+-------+---------+-------+----------+--------+---------+-----+-----+-------------+----------+------------------+---------------+------------+

+------------------+--

### Join The Game Dataframe With The Real Time Score Dataframe

In [19]:
cleanedDF = rtscoresDF.join(gameDF, rtscoresDF["key"] == gameDF["key"], "inner").drop(gameDF["key"])
print("cleanedDF : Cleaned Data Frame for use with ML algos")
cleanedDF.show(3)
print "Total Data Points in rtscoresDF = {0}".format(rtscoresDF.count())
print "Total Data Points in gameDF = {0}".format(gameDF.count())
print "Total Data Points in joined cleanedDF = {0}".format(cleanedDF.count())



lrDF : Logistic Regression Data Frame
+----------+--------+---------+------+-----------+------+-------------+--------------+---------+-----+-----+-------------+----------+-------------------+-----------+---------+-------+-----------+-----+-------+----------+------------------+-------+-------+------------+
|  dateOrig|      ts|teamlonga|scorea|  teamlongb|scoreb|   timestring|      timeleft|   gameid|teama|teamb|scorea-scoreb|      date|     score-div-time|teamaspread|overunder|teamaml|  teamlongh|teamh|teamhml|   dateStr|               key|fscorea|fscoreb|win-loss-enc|
+----------+--------+---------+------+-----------+------+-------------+--------------+---------+-----+-----+-------------+----------+-------------------+-----------+---------+-------+-----------+-----+-------+----------+------------------+-------+-------+------------+
|2016-04-20|00:01:44|  Memphis|    68|San Antonio|    94|(0:00 IN 4TH)|           0.0|400874376|  mem|  san|          -26|2016-04-19|              -26.0|  

### Save out cleanedDF for followon activities!

In [21]:
 cleanedDF.describe().toPandas()

,summary,ts,teamlonga,scorea,teamlongb,scoreb,timestring,timeleft,gameid,teama,...,overunder,teamaml,teamlongh,teamh,teamhml,dateStr,key,fscorea,fscoreb,win-loss-enc
0,count,13544,13544,13544,13544,13544,13544,13544,13544,13544,...,13544,13544,13544,13544,13544,13544,13544,13544,13544,13544
1,mean,None,None,53.12728883638511,None,56.24424099232132,None,22.408939998030966,4.008397731416125E8,None,...,204.7699965544398,16.427323291986575,None,None,-111.34832151998425,None,None,99.69240992321323,104.9308180744241,0.36370348493797994
2,stddev,None,None,30.4415396003271,None,31.99507785697002,None,14.307379388238457,19236.70219546687,None,...,9.007478496254663,247.05648102157372,None,None,250.39664354049978,None,None,12.196762945195383,11.864155709175236,0.48108247527538817
3,min,00:00:03,Atlanta,0,Atlanta,0,(0:00 IN 1ST),0.0,400829041,atl,...,180.5,-553.3333333333334,Atlanta,atl,-750.0,2016-04-05,2016-04-05.cha.tor,68,80,0.0
4,max,23:59:51,Washington,131,Washington,144,(HALFTIME),48.0,400874419,wsh,...,225.25,541.6666666666666,Washington,wsh,410.0,2016-04-24,2016-04-24.san.mem,131,144,1.0


In [26]:
# Since the data is small, you can coalesce to a single partition
# cleanedDF.coalesce(1).write.partitionBy('teama').mode('overwrite').format("csv").save("/data2/nba-rt-prediction/sparkfiles/cleanedDF")
cleanedDF.coalesce(1).write.mode('overwrite').format("csv").save("/data2/nba-rt-prediction/sparkfiles/cleanedDF")